# Data Deduplication and Cleaning Techniques

This notebook covers multiple methods to identify and remove duplicate or similar records using techniques such as hashing, fuzzy matching, cosine similarity, and clustering.

## 1. BASIC METHOD

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("complex_sales_data.csv")


# Check for duplicate records
print("Exact Duplicates:", df.duplicated().sum())
print("Duplicate Order_IDs:", df.duplicated(subset=['Order_ID']).sum())
print("Duplicate Customer Orders:", df.duplicated(subset=['Customer_Name', 'Product_Name', 'Order_Date']).sum())


Exact Duplicates: 1
Duplicate Order_IDs: 5
Duplicate Customer Orders: 2


In [32]:
# Trim spaces and convert names to lowercase
df["Customer_Name"] = df["Customer_Name"].str.strip().str.lower()

# Standardize date format
df["Order_Date"] = pd.to_datetime(df["Order_Date"], errors="coerce", dayfirst=True)  # Handle different formats

# Display cleaned data
print(df.head())


   Order_ID       Customer_Name Product_Category Product_Name  Quantity  \
0  0844e15d      jonathan perez      Electronics   Smartphone         2   
1  b3472ab1      william nelson        Groceries         Eggs         5   
2  d1dbe108     lawrence arnold        Groceries        Bread         2   
3  42ec140d  dr. robert johnson           Beauty    Sunscreen         2   
4  c8266aea           juan snow             Toys         Doll         8   

   Price_Per_Unit  Total_Amount Order_Date  
0          435.64        871.28 2024-11-24  
1          132.37        661.85 2023-10-29  
2          237.93        475.86 2024-05-15  
3          249.63        499.26 2023-05-31  
4          450.45       3603.60 2024-05-17  


C:\Users\Divyank\AppData\Local\Temp\ipykernel_23672\3754638451.py:5: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["Order_Date"] = pd.to_datetime(df["Order_Date"], errors="coerce", dayfirst=True)  # Handle different formats


In [33]:
df = df.drop_duplicates()
print("After removing exact duplicates:", df.shape)


After removing exact duplicates: (9, 8)


# Advance methods

## 1. Hash-Based Deduplication


USING HASH METHOD ON "ID" COLUMN AND "GAME_TITLE" COLUMN BECAUSE THERE ARE MULTIPLE SAME GAME TITLES FOR EACH "ID". 

### Steam video game Dataset

In [2]:
import pandas as pd
data = pd.read_csv("steam-200k.csv")
data.columns = ["id","game_title","use_type","play_hr","x"]

data.head(10)

,id,game_title,use_type,play_hr,x
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0
5,151603712,Fallout New Vegas,purchase,1.0,0
6,151603712,Fallout New Vegas,play,12.1,0
7,151603712,Left 4 Dead 2,purchase,1.0,0
8,151603712,Left 4 Dead 2,play,8.9,0
9,151603712,HuniePop,purchase,1.0,0


In [34]:
import hashlib

# Define columns to hash
cols_to_hash = ['id', 'game_title']

# Generate hash for each row based on specific columns
data['hash'] = data.apply(lambda x: hashlib.md5(''.join(str(x[col]) for col in cols_to_hash).encode()).hexdigest(), axis=1)

# Sort so that 'play' comes before 'purchase'
data.sort_values(by='use_type', ascending=True, inplace=True)

# Count duplicates (excluding the first occurrence, which is 'play' due to sorting)
duplicate_count = data.duplicated(subset=['hash']).sum()
print(f"Total duplicates found: {duplicate_count}")

# Drop duplicates, keeping the 'play' version
data = data.drop_duplicates(subset=['hash']).drop(columns=['hash'])


data.sort_values('id',inplace=True)
data.reset_index(drop=True, inplace=True)


Total duplicates found: 71195


In [68]:
# Show the result
pd.DataFrame(data.head(10))

,id,game_title,use_type,play_hr,x
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0
5,151603712,Fallout New Vegas,purchase,1.0,0
6,151603712,Fallout New Vegas,play,12.1,0
7,151603712,Left 4 Dead 2,purchase,1.0,0
8,151603712,Left 4 Dead 2,play,8.9,0
9,151603712,HuniePop,purchase,1.0,0


In [72]:
data.loc[data['id'] == data['id'][0]]


,id,game_title,use_type,play_hr,x
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0
...,...,...,...,...,...
60,151603712,HuniePop Original Soundtrack,purchase,1.0,0
61,151603712,The Banner Saga - Mod Content,purchase,1.0,0
62,151603712,The Elder Scrolls V Skyrim - Dawnguard,purchase,1.0,0
63,151603712,The Elder Scrolls V Skyrim - Dragonborn,purchase,1.0,0


In [ ]:
# Check for duplicate records
print("Exact Duplicates:", data.duplicated().sum())
# Check for duplicate game_title values within each id group
duplicates_within_id = data.duplicated(subset=['id', 'game_title'])
print("Duplicate game_title within each id:", duplicates_within_id.sum())

Exact Duplicates: 0
Duplicate game_title within each id: 0


## 2. Cosine Similarity for Text Deduplication

### IGN game dataset

In [2]:
import pandas as pd

data = pd.read_csv("ign.csv")
data = data.dropna()
data = data.drop("Unnamed: 0",axis =1,errors='ignore')

In [3]:
data

,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day
0,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,Y,2012,9,12
1,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,Y,2012,9,12
2,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,N,2012,9,12
3,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,N,2012,9,11
4,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,N,2012,9,11
...,...,...,...,...,...,...,...,...,...,...
18620,Good,Tokyo Mirage Sessions #FE,/games/fire-emblem-x-shin-megami-tensei/wii-u-...,Wii U,7.6,RPG,N,2016,6,29
18621,Amazing,LEGO Star Wars: The Force Awakens,/games/lego-star-wars-the-force-awakens/ps4-20...,PlayStation 4,9.0,"Action, Adventure",Y,2016,6,29
18622,Mediocre,Star Ocean: Integrity and Faithlessness,/games/star-ocean-5/ps4-20035681,PlayStation 4,5.8,RPG,N,2016,6,28
18623,Masterpiece,Inside,/games/inside-playdead/xbox-one-121435,Xbox One,10.0,Adventure,Y,2016,6,28


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Load the data
df = pd.read_csv("ign.csv")

# Combine 'title' and 'platform' to create a base for similarity comparison
df['combined_text'] = df['title'].astype(str) + " " + df['platform'].astype(str)

# Convert the combined text into TF-IDF vectors
vectorizer = TfidfVectorizer().fit_transform(df['combined_text'])
tfidf_matrix = vectorizer.toarray()

# Compute cosine similarity between all rows
cos_sim = cosine_similarity(tfidf_matrix)

# Set diagonal to 0 to ignore self-matching
np.fill_diagonal(cos_sim, 0)

# Set a similarity threshold (e.g. 0.9 = highly similar)
threshold = 0.85
duplicates = set()

# Store actual duplicate records for review
duplicate_records = []

for i in range(len(cos_sim)):
    if i in duplicates:
        continue
    similar_indices = np.where(cos_sim[i] > threshold)[0]
    for j in similar_indices:
        duplicates.add(j)
        duplicate_records.append((i, j))  # (original, duplicate)

# Print out a limited number of duplicate pairs for inspection
print(f"\nDetected {len(duplicate_records)} duplicate pairs (cosine similarity > {threshold}):\n")
for idx, (orig, dup) in enumerate(duplicate_records):
    if idx >= 20:
        print(f"... and {len(duplicate_records) - 20} more duplicate pairs not shown.")
        break
    print(f"[ORIGINAL #{orig}] {df.loc[orig, 'title']} ({df.loc[orig, 'platform']})")
    print(f"[DUPLICATE #{dup}] {df.loc[dup, 'title']} ({df.loc[dup, 'platform']})\n")

# Drop duplicate rows
df_deduplicated = df.drop(index=duplicates).reset_index(drop=True)

# Final result
print(f"\n✅ Removed {len(duplicates)} duplicates. New DataFrame shape: {df_deduplicated.shape}")
# Optional: Save the result
# df_deduplicated.to_csv("deduplicated_games.csv", index=False)



Detected 6836 duplicate pairs (cosine similarity > 0.85):

[ORIGINAL #3] NHL 13 (Xbox 360)
[DUPLICATE #4] NHL 13 (PlayStation 3)

[ORIGINAL #5] Total War Battles: Shogun (Macintosh)
[DUPLICATE #9] Total War Battles: Shogun (PC)

[ORIGINAL #6] Double Dragon: Neon (Xbox 360)
[DUPLICATE #8] Double Dragon: Neon (PlayStation 3)

[ORIGINAL #7] Guild Wars 2 (PC)
[DUPLICATE #6872] Guild Wars (PC)

[ORIGINAL #10] Tekken Tag Tournament 2 (PlayStation 3)
[DUPLICATE #11] Tekken Tag Tournament 2 (Xbox 360)

[ORIGINAL #10] Tekken Tag Tournament 2 (PlayStation 3)
[DUPLICATE #2595] Tekken Tag Tournament (PlayStation 2)

[ORIGINAL #13] Mark of the Ninja (Xbox 360)
[DUPLICATE #14] Mark of the Ninja (PC)

[ORIGINAL #15] Home: A Unique Horror Adventure (Macintosh)
[DUPLICATE #16] Home: A Unique Horror Adventure (PC)

[ORIGINAL #18] Way of the Samurai 4 (PlayStation 3)
[DUPLICATE #3801] Way of the Samurai (PlayStation 2)

[ORIGINAL #18] Way of the Samurai 4 (PlayStation 3)
[DUPLICATE #5915] Way of the Sam

In [5]:
# Check if any duplicated titles still exist in cleaned version
still_duplicated = df_deduplicated[df_deduplicated.duplicated(subset=['title', 'platform'], keep=False)]
print("Still duplicated after cleanup:", len(still_duplicated))

Still duplicated after cleanup: 0


In [ ]:
# Filter only valid indices that are within bounds
valid_indices = [idx for idx in duplicates if idx < len(data)]

# Now select rows using iloc
cosine_dups_df = data.iloc[valid_indices]

# Display results
print(f"Found {len(cosine_dups_df)} near-duplicate rows (via cosine similarity):\n")
print(cosine_dups_df[['title', 'platform', 'score', 'score_phrase']].head(20))


In [7]:
df_deduplicated

,Unnamed: 0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day,combined_text
0,0,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,Y,2012,9,12,LittleBigPlanet PS Vita PlayStation Vita
1,1,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,Y,2012,9,12,LittleBigPlanet PS Vita -- Marvel Super Hero E...
2,2,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,N,2012,9,12,Splice: Tree of Life iPad
3,3,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,N,2012,9,11,NHL 13 Xbox 360
4,4,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,N,2012,9,11,NHL 13 PlayStation 3
...,...,...,...,...,...,...,...,...,...,...,...,...
13719,18619,Good,Human Fall Flat,/games/human-fall-flat/pc-20051928,PC,7.9,"Puzzle, Action",N,2016,7,28,Human Fall Flat PC
13720,18620,Good,Tokyo Mirage Sessions #FE,/games/fire-emblem-x-shin-megami-tensei/wii-u-...,Wii U,7.6,RPG,N,2016,6,29,Tokyo Mirage Sessions #FE Wii U
13721,18621,Amazing,LEGO Star Wars: The Force Awakens,/games/lego-star-wars-the-force-awakens/ps4-20...,PlayStation 4,9.0,"Action, Adventure",Y,2016,6,29,LEGO Star Wars: The Force Awakens PlayStation 4
13722,18622,Mediocre,Star Ocean: Integrity and Faithlessness,/games/star-ocean-5/ps4-20035681,PlayStation 4,5.8,RPG,N,2016,6,28,Star Ocean: Integrity and Faithlessness PlaySt...


## 3. Rapidfuzz


Not working on the combination of the "title" and "platform" column as it is not detecting any duplicates as each combination is unique

In [4]:
import pandas as pd
from rapidfuzz import process, fuzz

# Load your dataset
data = pd.read_csv("ign.csv")  # or use df = pd.DataFrame(...) directly

def standardize_titles(data, threshold=90):
    data = data.copy()
    
    # Step 1: Combine title, platform, and year into a single column for fuzzy matching
    data['title_platform_year'] = data['title'] + ' - ' + data['platform'] + ' - ' + data['release_year'].astype(str)
    
    unique_combos = data['title_platform_year'].unique()
    mapping = {}

    # Step 2: Fuzzy match each title-platform-year combo
    for combo in unique_combos:
        match = process.extractOne(combo, unique_combos, scorer=fuzz.token_sort_ratio)
        if match and match[1] >= threshold and match[0] != combo:
            mapping[combo] = match[0]

    # Step 3: Replace close matches
    data['title_platform_year'] = data['title_platform_year'].replace(mapping)

    # Step 4: Split back into original columns
    split_cols = data['title_platform_year'].str.split(' - ', n=2, expand=True)
    data['title'] = split_cols[0]
    data['platform'] = split_cols[1]
    data['year'] = split_cols[2]

    # Optional: Drop the helper column
    data.drop(columns=['title_platform_year'], inplace=True)

    return data, mapping

# Apply the function
deduped_df, duplicates_mapping = standardize_titles(data, threshold=75)

# Show cleaned data
print("\n✅ Cleaned Data (sample):")
print(deduped_df.head(10))

# Show standardized duplicate mappings
print("\n🗂️ Standardized Duplicates:")
for original, standardized in duplicates_mapping.items():
    print(f"'{original}' ➡️ '{standardized}'")



✅ Cleaned Data (sample):
   Unnamed: 0 score_phrase                                              title  \
0           0      Amazing                            LittleBigPlanet PS Vita   
1           1      Amazing  LittleBigPlanet PS Vita -- Marvel Super Hero E...   
2           2        Great                               Splice: Tree of Life   
3           3        Great                                             NHL 13   
4           4        Great                                             NHL 13   
5           5         Good                          Total War Battles: Shogun   
6           6        Awful                                Double Dragon: Neon   
7           7      Amazing                                       Guild Wars 2   
8           8        Awful                                Double Dragon: Neon   
9           9         Good                          Total War Battles: Shogun   

                                                 url          platform  score  \
0

In [ ]:
deduped_df.shape

(18625, 12)

# using all the columns 

In [5]:
import pandas as pd
from rapidfuzz import process, fuzz

# Load your dataset
data = pd.read_csv("ign.csv")  # or use df = pd.DataFrame(...) directly

def standardize_rows(data, threshold=90):
    data = data.copy()
    
    # Step 1: Combine all columns into a single string per row for fuzzy matching
    data['combined'] = data.astype(str).agg(' - '.join, axis=1)

    unique_combos = data['combined'].unique()
    mapping = {}

    # Step 2: Fuzzy match each full-row string
    for combo in unique_combos:
        match = process.extractOne(combo, unique_combos, scorer=fuzz.token_sort_ratio)
        if match and match[1] >= threshold and match[0] != combo:
            mapping[combo] = match[0]

    # Step 3: Replace close matches
    data['combined'] = data['combined'].replace(mapping)

    # Step 4: Split combined string back into original columns
    split_cols = data['combined'].str.split(' - ', expand=True)
    split_cols.columns = data.columns.drop('combined')  # Set same column names (excluding helper)
    data = split_cols  # Replace original data

    return data, mapping

# Apply the function
deduped_df, duplicates_mapping = standardize_rows(data, threshold=75)

# Show cleaned data
print("\n✅ Cleaned Data (sample):")
print(deduped_df.head(10))

# Show standardized duplicate mappings
print("\n🗂️ Standardized Duplicates:")
for original, standardized in duplicates_mapping.items():
    print(f"'{original}' ➡️ '{standardized}'")


ValueError: Length mismatch: Expected axis has 13 elements, new values have 11 elements

# Applying only on title column

Not working properly as it is changing the names of the game title that have multiple sequences. Example "Fallout 2 ,3 " is converted to "Fallout" 

In [10]:
import pandas as pd
from rapidfuzz import process, fuzz

# Load dataset
data = pd.read_csv("ign.csv")

# Function to find and replace similar titles
def find_similar_titles(data, threshold=90):
    unique_titles = data['title'].unique().tolist()
    mapping = {}
    seen = set()

    for title in unique_titles:
        if title in seen:
            continue
        matches = process.extract(title, unique_titles, scorer=fuzz.token_sort_ratio, limit=5)
        for match_title, score, _ in matches:
            if match_title != title and score >= threshold and match_title not in seen:
                mapping[match_title] = title
                seen.add(match_title)
        seen.add(title)

    print(f"🔁 Found {len(mapping)} similar title pairs.\n")

    # Apply mapping to standardize titles
    data['title'] = data['title'].replace(mapping)
    return data, mapping

# Apply deduplication
deduped_df, title_mapping = find_similar_titles(data, threshold=90)

# Display results
print("🧹 Sample Cleaned Data:")
print(deduped_df.head(10))

print("\n🗂️ Showing First 20 Duplicate Title Mappings:")
for i, (original, replacement) in enumerate(title_mapping.items()):
    if i >= 20:
        print(f"... and {len(title_mapping) - 20} more")
        break
    print(f"'{original}' ➡️ '{replacement}'")


🔁 Found 1063 similar title pairs.

🧹 Sample Cleaned Data:
   Unnamed: 0 score_phrase                                              title  \
0           0      Amazing                            LittleBigPlanet PS Vita   
1           1      Amazing  LittleBigPlanet PS Vita -- Marvel Super Hero E...   
2           2        Great                               Splice: Tree of Life   
3           3        Great                                             NHL 13   
4           4        Great                                             NHL 13   
5           5         Good                          Total War Battles: Shogun   
6           6        Awful                                Double Dragon: Neon   
7           7      Amazing                                       Guild Wars 2   
8           8        Awful                                Double Dragon: Neon   
9           9         Good                          Total War Battles: Shogun   

                                                 u

In [8]:
import pandas as pd

# Load your dataset
data = pd.read_csv("ign.csv")  # or use your actual DataFrame

# Group by title and platform, then filter groups with more than 1 entry
duplicate_title_platform = data.groupby(['title', 'platform']).filter(lambda x: len(x) > 1)

# Sort for easier viewing (optional)
duplicate_title_platform = duplicate_title_platform.sort_values(by=['title', 'platform'])

# Display result
print("\n🔍 Titles with same platform appearing more than once:")
pd.DataFrame(duplicate_title_platform)



🔍 Titles with same platform appearing more than once:


,Unnamed: 0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day
6100,6100,Okay,Aladdin,/games/aladdin/gba-566703,Game Boy Advance,6.5,Platformer,N,2004,10,5
6211,6211,Okay,Aladdin,/games/aladdin/gba-566703,Game Boy Advance,6.5,Platformer,N,2004,10,5
12025,12025,Painful,Big League Sports,/games/big-league-sports/wii-14275098,Wii,2.3,"Sports, Compilation",N,2008,11,25
12117,12117,Painful,Big League Sports,/games/big-league-sports/wii-14275098,Wii,2.3,"Sports, Compilation",N,2008,11,25
14553,14553,Good,Blur,/games/blur/xbox-360-14222096,Xbox 360,7.0,Racing,N,2010,5,25
...,...,...,...,...,...,...,...,...,...,...,...
599,599,Amazing,WipEout XL,/games/wipeout-2097/ps-473,PlayStation,9.0,Racing,Y,1996,11,26
5594,5594,Good,Yu-Gi-Oh! World Championship Tournament 2004,/games/yu-gi-oh-world-championship-tournament-...,Game Boy Advance,7.5,"Card, Battle",N,2004,2,13
5604,5604,Good,Yu-Gi-Oh! World Championship Tournament 2004,/games/yu-gi-oh-world-championship-tournament-...,Game Boy Advance,7.5,"Card, Battle",N,2004,2,13
3887,3887,Mediocre,ZooCube,/games/zoocube/gba-479813,Game Boy Advance,5.3,Puzzle,N,2002,6,11


In [11]:
import pandas as pd

# Load your dataset
data = pd.read_csv("ign.csv")  # or use your DataFrame directly

# Step 1: Find titles that appear more than once
title_counts = data['title'].value_counts()
duplicate_titles = title_counts[title_counts > 1].index.tolist()

# Step 2: Filter data to only those titles
duplicates_df = data[data['title'].isin(duplicate_titles)]

# Step 3: Group by title and list all platforms
grouped = duplicates_df.groupby('title')['platform'].unique().reset_index()

# Step 4: Print result
print("\n🎮 Titles appearing more than once with their platforms:")
for _, row in grouped.iterrows():
    title = row['title']
    platforms = ", ".join(row['platform'])
    print(f"- {title}: {platforms}")



🎮 Titles appearing more than once with their platforms:
- 007 Legends: Xbox 360, PlayStation 3, Wii U
- 007: The World is Not Enough: Nintendo 64, PlayStation
- 1 vs. 100: Wireless, Nintendo DS
- 1001 Spikes: Wii U, PlayStation 4, PC, Nintendo 3DS, PlayStation Vita
- 102 Dalmatians: Puppies to the Rescue: PlayStation, Dreamcast, Game Boy Color
- 1080° Snowboarding: Nintendo 64, Wii
- 18-Wheeler American Pro Trucker: Dreamcast, GameCube
- 187 Ride or Die: PlayStation 2, Xbox
- 1942: Game Boy Color, Wireless
- 1942: Joint Strike: PlayStation 3, Xbox 360
- 2002 FIFA World Cup: PC, GameCube, Xbox, PlayStation 2
- 2006 FIFA World Cup: PlayStation Portable, Xbox 360, PC, Xbox, GameCube, PlayStation 2, Wireless, Nintendo DS
- 2010 FIFA World Cup South Africa: PlayStation Portable, Wii, PlayStation 3, Xbox 360
- 2014 FIFA World Cup Brazil: PlayStation 3, Xbox 360
- 24: Special Ops: Wireless, iPhone
- 24: The Game: Wireless, PlayStation 2
- 25 To Life: PC, PlayStation 2, Xbox
- 3 on 3 NHL Arca